In [1]:
import aframe as af
import pandas as pd
import findspark
findspark.init()
import pyspark
from pyspark.sql import Row
from pyspark.sql import SQLContext
from pyspark import SparkContext
from pyspark.sql.functions import *
import numpy as np

In [2]:
message_af = af.AFrame(dataverse='TinySocial', dataset='GleambookMessages')
user_af = af.AFrame(dataverse='TinySocial', dataset='GleambookUsers')
message_df = message_af.toPandas()
user_df = user_af.toPandas()

In [3]:
sc = SparkContext(appName="app")
sqlContext = SQLContext(sc)

In [4]:
message_spark = sqlContext.createDataFrame(message_df,message_df.columns.tolist())
user_spark = sqlContext.createDataFrame(user_df.fillna('None'),user_df.columns.tolist())

## Pandas DataFrame Functions

In [5]:
user_df.head()

,alias,employment,friendIds,gender,id,name,nickname,userSince
0,Margarita,"[{'organizationName': 'Codetechno', 'startDate...","[2, 3, 6, 10]",F,1,MargaritaStoddard,Mags,2012-08-20T10:10:00.000Z
1,Isbel,"[{'organizationName': 'Hexviafind', 'startDate...","[1, 4]",NaN,2,IsbelDull,Izzy,2011-01-22T10:10:00.000Z
2,Emory,"[{'organizationName': 'geomedia', 'startDate':...","[1, 5, 8, 9]",NaN,3,EmoryUnk,NaN,2012-07-10T10:10:00.000Z
3,Nicholas,"[{'organizationName': 'Zamcorporation', 'start...",[2],NaN,4,NicholasStroh,NaN,2010-12-27T10:10:00.000Z
4,Von,"[{'organizationName': 'Kongreen', 'startDate':...","[3, 6, 10]",NaN,5,VonKemble,NaN,2010-01-05T10:10:00.000Z


### Elementwise functions

In [6]:
f = lambda x: len(str(x))
user_df['name'].map(f)

0    17
1     9
2     8
3    13
4     9
5    11
6    14
7    12
8    14
9     9
Name: name, dtype: int64

In [7]:
user_df['alias'].map(f)

0    9
1    5
2    5
3    8
4    3
5    6
6    7
7    4
8    7
9    4
Name: alias, dtype: int64

### Tablewise functions

In [8]:
user_df.apply(f)

alias         176
employment    581
friendIds     220
gender        117
id            103
name          255
nickname      139
userSince     330
dtype: int64

## AFrame Functions

### Elementwise Functions

In [9]:
name_len = user_af['name'].map('length')

In [10]:
name_len.query

'select value length(t.name) from TinySocial.GleambookUsers t;'

In [11]:
name_len.collect()

,0
0,17
1,9
2,8
3,13
4,9
5,11
6,14
7,12
8,14
9,9


### Functions with arguments

In [12]:
name_contain = user_af['name'].map('contains', 'Suzan')

In [13]:
name_contain.query

'select value contains(t.name, "Suzan") from TinySocial.GleambookUsers t;'

In [14]:
name_contain.collect()

,0
0,False
1,False
2,False
3,False
4,False
5,False
6,True
7,False
8,False
9,False


### Tablewise Functions

In [15]:
fields = message_af.apply('get_object_fields')

In [16]:
fields.query

'select value get_object_fields(t) from TinySocial.GleambookMessages t;'

In [17]:
fields.head(1)

,0,1,2,3,4
0,"{'field-name': 'messageId', 'field-type': 'big...","{'field-name': 'authorId', 'field-type': 'bigi...","{'field-name': 'inResponseTo', 'field-type': '...","{'field-name': 'senderLocation', 'field-type':...","{'field-name': 'message', 'field-type': 'strin..."


### PySpark

In [18]:
user_spark.show()

+---------+--------------------+-------------+------+---+-----------------+--------+--------------------+
|    alias|          employment|    friendIds|gender| id|             name|nickname|           userSince|
+---------+--------------------+-------------+------+---+-----------------+--------+--------------------+
|Margarita|[[organizationNam...|[2, 3, 6, 10]|     F|  1|MargaritaStoddard|    Mags|2012-08-20T10:10:...|
|    Isbel|[[organizationNam...|       [1, 4]|  None|  2|        IsbelDull|    Izzy|2011-01-22T10:10:...|
|    Emory|[[organizationNam...| [1, 5, 8, 9]|  None|  3|         EmoryUnk|    None|2012-07-10T10:10:...|
| Nicholas|[[organizationNam...|          [2]|  None|  4|    NicholasStroh|    None|2010-12-27T10:10:...|
|      Von|[[organizationNam...|   [3, 6, 10]|  None|  5|        VonKemble|    None|2010-01-05T10:10:...|
|   Willis|[[organizationNam...|    [1, 3, 7]|  None|  6|      WillisWynne|    None|2005-01-17T10:10:...|
|  Suzanna|[[organizationNam...|          [6]|

In [19]:
user_spark.select(explode(user_spark['friendIds'])).show()

+---+
|col|
+---+
|  2|
|  3|
|  6|
| 10|
|  1|
|  4|
|  1|
|  5|
|  8|
|  9|
|  2|
|  3|
|  6|
| 10|
|  1|
|  3|
|  7|
|  6|
|  3|
|  3|
+---+
only showing top 20 rows



In [20]:
user_spark.withColumn('friend', explode('friendIds')).show(2)

+---------+--------------------+-------------+------+---+-----------------+--------+--------------------+------+
|    alias|          employment|    friendIds|gender| id|             name|nickname|           userSince|friend|
+---------+--------------------+-------------+------+---+-----------------+--------+--------------------+------+
|Margarita|[[organizationNam...|[2, 3, 6, 10]|     F|  1|MargaritaStoddard|    Mags|2012-08-20T10:10:...|     2|
|Margarita|[[organizationNam...|[2, 3, 6, 10]|     F|  1|MargaritaStoddard|    Mags|2012-08-20T10:10:...|     3|
+---------+--------------------+-------------+------+---+-----------------+--------+--------------------+------+
only showing top 2 rows



### AFrame Unnest Data

In [21]:
user_af.unnest(user_af['friendIds']).head()

,0
0,2
1,3
2,6
3,10
4,1


In [22]:
user_af.unnest(user_af['friendIds'], appended=True, name='friend').query

'select u friend, t.* from TinySocial.GleambookUsers t unnest t.friendIds u;'

In [23]:
user_af.unnest(user_af['friendIds'], appended=True, name='friendID').head()

,alias,employment,friendID,friendIds,gender,id,name,nickname,userSince
0,Margarita,"[{'organizationName': 'Codetechno', 'startDate...",2,"[2, 3, 6, 10]",F,1,MargaritaStoddard,Mags,2012-08-20T10:10:00.000Z
1,Margarita,"[{'organizationName': 'Codetechno', 'startDate...",3,"[2, 3, 6, 10]",F,1,MargaritaStoddard,Mags,2012-08-20T10:10:00.000Z
2,Margarita,"[{'organizationName': 'Codetechno', 'startDate...",6,"[2, 3, 6, 10]",F,1,MargaritaStoddard,Mags,2012-08-20T10:10:00.000Z
3,Margarita,"[{'organizationName': 'Codetechno', 'startDate...",10,"[2, 3, 6, 10]",F,1,MargaritaStoddard,Mags,2012-08-20T10:10:00.000Z
4,Isbel,"[{'organizationName': 'Hexviafind', 'startDate...",1,"[1, 4]",NaN,2,IsbelDull,Izzy,2011-01-22T10:10:00.000Z


### Arithmetic Operations

In [24]:
(user_af['id'] + 3).query

'select value t.id + 3 from TinySocial.GleambookUsers t;'

In [25]:
user_af['id'].add(3).head()

,0
0,4
1,5
2,6
3,7
4,8


In [26]:
user_af['id'].mul(3).head()

,0
0,3
1,6
2,9
3,12
4,15


## Persisting Data in AFrame

In [27]:
message_af = af.AFrame(dataverse='TinySocial', dataset='GleambookMessages')
user_af = af.AFrame(dataverse='TinySocial', dataset='GleambookUsers')

In [28]:
user_af.toPandas().head(2)

,alias,employment,friendIds,gender,id,name,nickname,userSince
0,Margarita,"[{'organizationName': 'Codetechno', 'startDate...","[2, 3, 6, 10]",F,1,MargaritaStoddard,Mags,2012-08-20T10:10:00.000Z
1,Isbel,"[{'organizationName': 'Hexviafind', 'startDate...","[1, 4]",NaN,2,IsbelDull,Izzy,2011-01-22T10:10:00.000Z


In [29]:
tmp = user_af.withColumn('id_3', user_af['id']+3)
tmp.head()

,alias,employment,friendIds,gender,id,id_3,name,nickname,userSince
0,Margarita,"[{'organizationName': 'Codetechno', 'startDate...","[2, 3, 6, 10]",F,1,4,MargaritaStoddard,Mags,2012-08-20T10:10:00.000Z
1,Isbel,"[{'organizationName': 'Hexviafind', 'startDate...","[1, 4]",NaN,2,5,IsbelDull,Izzy,2011-01-22T10:10:00.000Z
2,Emory,"[{'organizationName': 'geomedia', 'startDate':...","[1, 5, 8, 9]",NaN,3,6,EmoryUnk,NaN,2012-07-10T10:10:00.000Z
3,Nicholas,"[{'organizationName': 'Zamcorporation', 'start...",[2],NaN,4,7,NicholasStroh,NaN,2010-12-27T10:10:00.000Z
4,Von,"[{'organizationName': 'Kongreen', 'startDate':...","[3, 6, 10]",NaN,5,8,VonKemble,NaN,2010-01-05T10:10:00.000Z


In [42]:
tmp.query

'select u friendID, t.* from TinySocial.GleambookUsers t unnest t.friendIds u;'

In [31]:
new_af = tmp.persist('UserCopy')

In [32]:
type(new_af)

aframe.AFrame

In [33]:
new_af._dataverse

'_Temp'

In [34]:
new_af._dataset

'UserCopy'

In [35]:
new_af.toPandas()

,alias,employment,friendIds,gender,id,id_3,name,nickname,userSince
0,Margarita,"[{'organizationName': 'Codetechno', 'startDate...","[2, 3, 6, 10]",F,1,4,MargaritaStoddard,Mags,2012-08-20T10:10:00.000Z
1,Isbel,"[{'organizationName': 'Hexviafind', 'startDate...","[1, 4]",NaN,2,5,IsbelDull,Izzy,2011-01-22T10:10:00.000Z
2,Emory,"[{'organizationName': 'geomedia', 'startDate':...","[1, 5, 8, 9]",NaN,3,6,EmoryUnk,NaN,2012-07-10T10:10:00.000Z
3,Nicholas,"[{'organizationName': 'Zamcorporation', 'start...",[2],NaN,4,7,NicholasStroh,NaN,2010-12-27T10:10:00.000Z
4,Von,"[{'organizationName': 'Kongreen', 'startDate':...","[3, 6, 10]",NaN,5,8,VonKemble,NaN,2010-01-05T10:10:00.000Z
5,Willis,"[{'organizationName': 'jaydax', 'startDate': '...","[1, 3, 7]",NaN,6,9,WillisWynne,NaN,2005-01-17T10:10:00.000Z
6,Suzanna,"[{'organizationName': 'Labzatron', 'startDate'...",[6],NaN,7,10,SuzannaTillson,NaN,2012-08-07T10:10:00.000Z
7,Nila,"[{'organizationName': 'Plexlane', 'startDate':...",[3],NaN,8,11,NilaMilliron,NaN,2008-01-01T10:10:00.000Z
8,Woodrow,"[{'organizationName': 'Zuncan', 'startDate': '...","[3, 10]",NaN,9,12,WoodrowNehling,Woody,2005-09-20T10:10:00.000Z
9,Bram,"[{'organizationName': 'physcane', 'startDate':...","[1, 5, 9]",NaN,10,13,BramHatch,NaN,2010-10-16T10:10:00.000Z


In [36]:
af.AFrame.drop(new_af)

'success'

In [37]:
tmp = user_af.unnest(user_af['friendIds'], appended=True, name='friendID')
tmp.head(2)

,alias,employment,friendID,friendIds,gender,id,name,nickname,userSince
0,Margarita,"[{'organizationName': 'Codetechno', 'startDate...",2,"[2, 3, 6, 10]",F,1,MargaritaStoddard,Mags,2012-08-20T10:10:00.000Z
1,Margarita,"[{'organizationName': 'Codetechno', 'startDate...",3,"[2, 3, 6, 10]",F,1,MargaritaStoddard,Mags,2012-08-20T10:10:00.000Z


In [40]:
copy_af = tmp.persist('UserCopy')
copy_af.toPandas().head()

,alias,employment,friendID,friendIds,gender,id,name,nickname,userSince
0,Margarita,"[{'organizationName': 'Codetechno', 'startDate...",2,"[2, 3, 6, 10]",F,1,MargaritaStoddard,Mags,2012-08-20T10:10:00.000Z
1,Margarita,"[{'organizationName': 'Codetechno', 'startDate...",3,"[2, 3, 6, 10]",F,1,MargaritaStoddard,Mags,2012-08-20T10:10:00.000Z
2,Margarita,"[{'organizationName': 'Codetechno', 'startDate...",6,"[2, 3, 6, 10]",F,1,MargaritaStoddard,Mags,2012-08-20T10:10:00.000Z
3,Margarita,"[{'organizationName': 'Codetechno', 'startDate...",10,"[2, 3, 6, 10]",F,1,MargaritaStoddard,Mags,2012-08-20T10:10:00.000Z
4,Isbel,"[{'organizationName': 'Hexviafind', 'startDate...",1,"[1, 4]",NaN,2,IsbelDull,Izzy,2011-01-22T10:10:00.000Z


In [41]:
af.AFrame.drop(copy_af)

'success'